In [1]:
import csv
import json
import time
import datetime
import pprint
import pyperclip

def convert_time(s):
    s = s.replace('0217', '2017')
    s = s.replace('Director', '')
    if s:
        return time.mktime(datetime.datetime.strptime(s, "%Y-%m-%d").timetuple())
    else:
        return float('nan')
        

DESIRED_FIELDS = ['organization_name', 'organization_type'
                  , 'position_held', 'from_date', 'to_date', 'staffer_id', 'organization_id']


with open('outside_government_positions.csv', 'r') as f:
    reader = csv.DictReader(f)
    staffer_org_output = {}
    organization_output = {}
    organization_id_output = {}
    for row in reader:
        person = {k.strip():v for k, v in row.items() if k in DESIRED_FIELDS}
        person['from_date'] = convert_time(person['from_date'])
        person['to_date'] = convert_time(person['to_date'])
        
        staffer_org_output.setdefault(person['staffer_id'], []).append(person)
        
        organization_id_output.setdefault(person['organization_id'], []).append(person)
        
        organization_output.setdefault(person['organization_type'], {})
        organization_output[person['organization_type']].setdefault(person['organization_id'], []).append(person)
        

        
with open('organization.js', 'w') as f:
    f.write('const organizations = ' + json.dumps(organization_output, indent=4, sort_keys=True))
    
with open('staffer_org.js', 'w') as f:
    f.write('const staffer_orgs = ' + json.dumps(staffer_org_output, indent=4, sort_keys=True))
    
with open('organization_id.js', 'w') as f:
    f.write('const organzation_ids = ' + json.dumps(organization_id_output, indent=4, sort_keys=True))
    
#pyperclip.copy(str(organization_output))

